# Imports

In [22]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras import regularizers
import keras
import os

# Single View

## Creating variables

In [24]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
BATCH_SIZE = 16

trainData = pd.read_csv('excel/multi-view/csv/multi-view-train.csv')
trainDir = 'data-train-val-test-one-folder/train'

validationData = pd.read_csv('excel/multi-view/csv/multi-view-val.csv')
validationDir = 'data-train-val-test-one-folder/val/'

testData = pd.read_csv('excel/multi-view/csv/multi-view-test.csv')
testDir = 'data-train-val-test-one-folder/test/'

## Creating functions

In [25]:
trainGenerator = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1,
                         rotation_range=10,
                         shear_range = 0.1,
                         fill_mode='nearest',
                         horizontal_flip = True,
                         preprocessing_function = keras.applications.inception_v3.preprocess_input)

validationGenerator = ImageDataGenerator(preprocessing_function = keras.applications.inception_v3.preprocess_input)

testGenerator = ImageDataGenerator(preprocessing_function = keras.applications.inception_v3.preprocess_input)

def SetClassesAndParts(root):
    classes = os.listdir(root) # gets a list of all classes from data
    pathToParts = os.path.join(root, classes[0])
    parts = os.listdir(pathToParts) # gets a list of all parts from data
    return classes, parts

def create_model(baseModel, numberOfClasses):
    
    inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = baseModel(inputs, training = False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.3)(x)

    x = keras.layers.Flatten()(x)

    x = keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.L1L2())(x)
    outputs = keras.layers.Dense(numberOfClasses, activation='softmax')(x)

    model = keras.Model(inputs,outputs)

    return model

In [26]:
root = r'data'

classes, parts = SetClassesAndParts(root)

## Model

In [27]:
base_model = InceptionV3(weights = 'imagenet',
                        include_top = False,
                        input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

### Training and evaluation

In [29]:
epochs = 10


for part in parts:
    
    train_data = trainData[[part, 'class']]
    validation_data = validationData[[part, 'class']]
    test_data = testData[[part, 'class']]       
    
    train_data_generator = trainGenerator.flow_from_dataframe(train_data, directory = trainDir,
						       x_col = part, y_col = "class",
						       class_mode = "categorical", 
                               shuffle = True,
                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                               batch_size=BATCH_SIZE,
                               interpolation = 'lanczos')

    validation_data_generator = validationGenerator.flow_from_dataframe(validation_data, directory = validationDir,
                                x_col = part, y_col = "class",
                                class_mode = "categorical", 
                                shuffle = True,
                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                batch_size=BATCH_SIZE,
                                interpolation = 'lanczos')

    test_data_generator = testGenerator.flow_from_dataframe(test_data, directory = testDir,
                                x_col = part, y_col = "class",
                                class_mode = "categorical",
                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                interpolation = 'lanczos')
    
    nb_train_samples = train_data_generator.samples
    nb_validation_samples = validation_data_generator.samples

    model = create_model(base_model, len(classes))

    model.compile(Adam(0.001), 
                    loss='categorical_crossentropy', 
                    metrics=['accuracy']) 

    callbacks_list = [ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001),
                        EarlyStopping(monitor= "val_loss", patience=5),
                        ModelCheckpoint(f'models/single-view/model_{part}.h5',
                                        monitor = "val_loss",
                                        save_best_only=True, 
                                        mode='min', 
                                        verbose = 1)]

    history = model.fit(train_data_generator,
                        validation_data=validation_data_generator,
                        epochs = epochs,
                        callbacks=callbacks_list)


Found 224 validated image filenames belonging to 8 classes.
Found 64 validated image filenames belonging to 8 classes.
Found 32 validated image filenames belonging to 8 classes.


Epoch 1/10
14/14 [==============================] - ETA: 0s - loss: 2.3856 - accuracy: 0.2366
Epoch 1: val_loss improved from inf to 1.47924, saving model to models/single-view\model_back-panels.h5


c:\Users\73351374\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 [==============================] - 97s 7s/step - loss: 2.3856 - accuracy: 0.2366 - val_loss: 1.4792 - val_accuracy: 0.5625 - lr: 0.0010
Epoch 2/10
14/14 [==============================] - ETA: 0s - loss: 1.2006 - accuracy: 0.5536
Epoch 2: val_loss improved from 1.47924 to 1.00320, saving model to models/single-view\model_back-panels.h5
14/14 [==============================] - 86s 6s/step - loss: 1.2006 - accuracy: 0.5536 - val_loss: 1.0032 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 3/10
14/14 [==============================] - ETA: 0s - loss: 0.7340 - accuracy: 0.7455
Epoch 3: val_loss improved from 1.00320 to 0.96362, saving model to models/single-view\model_back-panels.h5
14/14 [==============================] - 86s 6s/step - loss: 0.7340 - accuracy: 0.7455 - val_loss: 0.9636 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 4/10
14/14 [==============================] - ETA: 0s - loss: 0.6843 - accuracy: 0.7902
Epoch 4: val_loss improved from 0.96362 to 0.65886, saving model to models/s

c:\Users\73351374\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 [==============================] - 98s 7s/step - loss: 1.8278 - accuracy: 0.4241 - val_loss: 1.1207 - val_accuracy: 0.6094 - lr: 0.0010
Epoch 2/10
14/14 [==============================] - ETA: 0s - loss: 0.7369 - accuracy: 0.7143
Epoch 2: val_loss improved from 1.12067 to 0.63231, saving model to models/single-view\model_front-panels.h5
14/14 [==============================] - 88s 6s/step - loss: 0.7369 - accuracy: 0.7143 - val_loss: 0.6323 - val_accuracy: 0.7656 - lr: 0.0010
Epoch 3/10
14/14 [==============================] - ETA: 0s - loss: 0.3950 - accuracy: 0.8750
Epoch 3: val_loss improved from 0.63231 to 0.60801, saving model to models/single-view\model_front-panels.h5
14/14 [==============================] - 88s 6s/step - loss: 0.3950 - accuracy: 0.8750 - val_loss: 0.6080 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 4/10
14/14 [==============================] - ETA: 0s - loss: 0.4323 - accuracy: 0.8348
Epoch 4: val_loss improved from 0.60801 to 0.36049, saving model to models

c:\Users\73351374\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 [==============================] - 93s 6s/step - loss: 2.2182 - accuracy: 0.3438 - val_loss: 1.2064 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/10
14/14 [==============================] - ETA: 0s - loss: 1.0033 - accuracy: 0.6473
Epoch 2: val_loss improved from 1.20642 to 0.68158, saving model to models/single-view\model_side-hinges.h5
14/14 [==============================] - 84s 6s/step - loss: 1.0033 - accuracy: 0.6473 - val_loss: 0.6816 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/10
14/14 [==============================] - ETA: 0s - loss: 0.5864 - accuracy: 0.8080
Epoch 3: val_loss improved from 0.68158 to 0.67774, saving model to models/single-view\model_side-hinges.h5
14/14 [==============================] - 84s 6s/step - loss: 0.5864 - accuracy: 0.8080 - val_loss: 0.6777 - val_accuracy: 0.7188 - lr: 0.0010
Epoch 4/10
14/14 [==============================] - ETA: 0s - loss: 0.4852 - accuracy: 0.8438
Epoch 4: val_loss did not improve from 0.67774
14/14 [======================